In [1]:
import tarski
import tarski.evaluators
from tarski.theories import Theory
from tarski.io.fstrips import ParsingError, FstripsReader
from tarski.syntax import *

# Parsing and Processing PDDL

In [2]:
reader = FstripsReader(raise_on_error=True)

In [3]:
reader.parse_domain('probBLOCKS-domain.pddl')
reader.parse_instance('probBLOCKS-4-2.pddl')

In [4]:
L = reader.problem.language

We can retrieve any sort (or _type_ in PDDL-parlance), constant, function or predicate 
via the `get` method of the class `Language`. Since we are dealing
with the classic `Blocks World` benchmark we know there must be predicates `on(x,y)` and `clear(x)` 

In [5]:
on = L.get('on')
clear = L.get('clear')

`Language` instances allow us to iterate over the list of declared predicates etc. via the
properties `predicates`, `functions`

In [6]:
for pred in L.predicates:
    print(str(pred))

=/2
!=/2
on/2
ontable/1
clear/1
handempty/0
holding/1


As seen above `Tarski` parser defines the equality predicate and its negation, as  well as the
usual `on` etc. The number after the name of the predicate indicates its _arity_ (i.e. number of arguments).

Getting our hands on the objects defined in the instance file `probBLOCKS-4-2.pddl` requires to 
navigate the type hierarchy. First, we can take a look at what are the sorts defined similarly as
we do for predicates

In [7]:
for S in L.sorts:
    print(str(S))

Sort(object)


There is only one sort defined, `object`. We can check out its elements 

In [8]:
objects = L.get('object')
for obj in objects.domain():
    print(str(obj), type(obj))

c <class 'tarski.syntax.terms.Constant'>
d <class 'tarski.syntax.terms.Constant'>
b <class 'tarski.syntax.terms.Constant'>
a <class 'tarski.syntax.terms.Constant'>


### Excursion: Indexing Logical Elements

It is handy to have the objects indexed in a table for easy retrieval. We can have such a table
by name

In [9]:
obj_table1 = { str(obj): obj for obj in objects.domain()}
obj_table1

{'a': a (object), 'b': b (object), 'c': c (object), 'd': d (object)}

Alternatively, we can use the constants themselves as keys in a dictionary, by wrapping them with
an adapter in this way

In [10]:
obj_table2 = { symref(obj): [] for obj in objects.domain()}
obj_table2

{symref[a]: [], symref[b]: [], symref[c]: [], symref[d]: []}

Wrapping constant, terms and atoms is necessary as `Tarski` overloads most of the standard Python 
operators to provide compact syntax for formulas and expressions. For instance, if we wanted
to compare whether two constants are the same _value_ one could try the following

In [11]:
x = L.get('a')
y = L.get('b')
x == y

=(a,b)

Yet using the comparison operator `==` results in the _atom_ `=(a,b)`, as `a` and `b` stand for
two elements of a first-order language. If we use the wrappers

In [12]:
x = L.get('a')
y = L.get('b')
symref(x) == symref(y)

False

Then we get to compare the actual _physical_ objects than the _logical elements_ (e.g.
 constants, arithmetic expressions, Boolean formulas) they represent.

## Grounding    

`Tarski` supports two different strategies for grounding lifted representations of STRIPS and ADL. One is based around
the idea by Malte Helmert of constructing a logic program and computing its models so as to determine the reachability
of a ground atom _online_. This is strategy constrasts with the classical approach, which we refer to as `naive`, where 
all potential groundings are first enumerated and then tested by reachability. While Helmert's approach is slower
for smaller problems, it does pay off greatly for instances with thousands of ground actions, or more, which can only
rarely be processed even by optimized implementations of the `naive` approach.

To implement Helmert's algorithm, or `lp` from now on, we rely on the excellent tools developed by for Answer Set
Programming, and inparticular, `gringo`, the suite of grounding tools for
disjunctivel logic programs. Stable versions of tools are neatly packaged in Ubuntu 18.04 so you can install them
like this

```
$ sudo apt install gringo
```

We can get hold of the classes encapsulating the grounding process from the `tarski.grounding` module

In [13]:
from tarski.grounding import LPGroundingStrategy
from tarski.grounding.errors import ReachabilityLPUnsolvable

and ground the instance of Blocks World with a one-liner

In [14]:
try:
    grounding = LPGroundingStrategy(reader.problem)
except ReachabilityLPUnsolvable:
    print("Problem was determined to be unsolvable during grounding")

Note that the `lp` grounder can determine if an instance is unsolvable, so we need to handle that possible exception.

In [15]:
actions = grounding.ground_actions()

for name, ops in actions.items():
    print('Action schema', name, 'got', len(ops), 'ground actions')

Action schema pick-up got 4 ground actions
Action schema put-down got 4 ground actions
Action schema stack got 16 ground actions
Action schema unstack got 16 ground actions


In [16]:
lpvariables = grounding.ground_state_variables()
for atom_index, atoms in lpvariables.enumerate():
    print('p_{}:'.format(atom_index), atoms)

p_0: clear(b)
p_1: clear(c)
p_2: clear(a)
p_3: clear(d)
p_4: on(d,b)
p_5: on(c,b)
p_6: on(b,b)
p_7: on(a,d)
p_8: on(d,d)
p_9: on(a,c)
p_10: on(d,a)
p_11: on(c,d)
p_12: on(d,c)
p_13: on(c,a)
p_14: on(c,c)
p_15: on(b,c)
p_16: on(a,a)
p_17: on(b,a)
p_18: on(b,d)
p_19: on(a,b)
p_20: handempty()
p_21: holding(b)
p_22: holding(c)
p_23: holding(a)
p_24: holding(d)
p_25: ontable(b)
p_26: ontable(c)
p_27: ontable(a)
p_28: ontable(d)


## The $K_0$ compilation

For the purpose of this tutorial we will look at the seminal paper by Palacios and Geffner
"Compiling Uncertainty Away in Conformant Planning Problems with Bounded Width", and see how
we can implement the $K_0$ compilation of conformant into classical problems.

*Definition* (Translation $K_0$). For a conformant planning problem $P=\langle F,I,O,G\rangle$, the 
translation $K_0(P) = \langle F', I', O', G'\rangle$ is classical planning problem with
 - $F' = \{ KL, K\neg L\, \mid\, L \in F \}$
 - $I' = \{ KL \, \mid \, L \text{ is a unit clause in } I\}$
 - $G' = \{ KL \, \mid \, L \in G\}$
 - $O' = O$ but with each precondition $L$ for $a \in O$ replaced by $KL$, and each conditional effect 
 $a: C \rightarrow L$ replaced by $a: KC \rightarrow KL$ and $a: \neg K \neg C \rightarrow \neg K \neg L$.

### Constructing the fluent set $F'$

In [17]:
KP_lang = tarski.language("K0(P)", theories=[Theory.EQUALITY])

In [18]:
p_lits = KP_lang.sort("P-literals", KP_lang.Object)

In [19]:
F_lits = []
P_lits = []
p_lits_positive = []
p_lits_negative = []
for atom_index, atoms in lpvariables.enumerate():
    f_atom = Atom(atoms.symbol, atoms.binding)
    fp_lit = f_atom
    fn_lit = neg(f_atom)
    p_lits_positive += [KP_lang.constant(str(fp_lit), p_lits)]
    p_lits_negative += [KP_lang.constant(str(fn_lit), p_lits)]
    F_lits += [fp_lit, fn_lit]
    P_lits += [p_lits_positive[-1], p_lits_negative[-1]]

In [20]:
K = KP_lang.predicate("K", p_lits)

### Constructing the initial state $I'$

In [21]:
reader.problem.init.evaluator = tarski.evaluators.simple.evaluate

In [22]:
I = []
for atom_index, atoms in lpvariables.enumerate():
    atom = Atom(atoms.symbol, atoms.binding)
    if reader.problem.init[atom]:
        I += [atom]
    else:
        I += [neg(atom)]

In [23]:
for p in I:
    print(p)


(not clear(b))
clear(c)
clear(a)
clear(d)
(not on(d,b))
on(c,b)
(not on(b,b))
(not on(a,d))
(not on(d,d))
(not on(a,c))
(not on(d,a))
(not on(c,d))
(not on(d,c))
(not on(c,a))
(not on(c,c))
(not on(b,c))
(not on(a,a))
(not on(b,a))
(not on(b,d))
(not on(a,b))
handempty()
(not holding(b))
(not holding(c))
(not holding(a))
(not holding(d))
ontable(b)
(not ontable(c))
ontable(a)
ontable(d)


In [24]:
handempty = reader.problem.language.get('handempty')
holding = reader.problem.language.get('holding')
a, b, c, d = reader.problem.language.get('a', 'b', 'c', 'd')

In [25]:
I = [handempty(), neg(holding(a)), neg(holding(b)), neg(holding(c)), neg(holding(d))]

In [26]:
covered_p_lits = set()
K_I = []
for l0 in I:
    p_l0 = KP_lang.get(str(l0))
    K_I += [K(p_l0)]
    covered_p_lits.add(symref(l0))
for atom_index, atoms in lpvariables.enumerate():
    lp = Atom(atoms.symbol, atoms.binding)
    p_lp = KP_lang.get(str(lp))
    ln = neg(lp)
    p_ln = KP_lang.get(str(ln))
    if lp not in covered_p_lits:
        K_I += [neg(K(p_lp))]
    if ln not in covered_p_lits:
        K_I += [neg(K(p_ln))]

In [27]:
for k_p in K_I:
    print(k_p)

K(handempty())
K((not holding(a)))
K((not holding(b)))
K((not holding(c)))
K((not holding(d)))
(not K(clear(b)))
(not K((not clear(b))))
(not K(clear(c)))
(not K((not clear(c))))
(not K(clear(a)))
(not K((not clear(a))))
(not K(clear(d)))
(not K((not clear(d))))
(not K(on(d,b)))
(not K((not on(d,b))))
(not K(on(c,b)))
(not K((not on(c,b))))
(not K(on(b,b)))
(not K((not on(b,b))))
(not K(on(a,d)))
(not K((not on(a,d))))
(not K(on(d,d)))
(not K((not on(d,d))))
(not K(on(a,c)))
(not K((not on(a,c))))
(not K(on(d,a)))
(not K((not on(d,a))))
(not K(on(c,d)))
(not K((not on(c,d))))
(not K(on(d,c)))
(not K((not on(d,c))))
(not K(on(c,a)))
(not K((not on(c,a))))
(not K(on(c,c)))
(not K((not on(c,c))))
(not K(on(b,c)))
(not K((not on(b,c))))
(not K(on(a,a)))
(not K((not on(a,a))))
(not K(on(b,a)))
(not K((not on(b,a))))
(not K(on(b,d)))
(not K((not on(b,d))))
(not K(on(a,b)))
(not K((not on(a,b))))
(not K(handempty()))
(not K((not handempty())))
(not K(holding(b)))
(not K((not holding(b))))
(no

### Constructing the goal state $G'$

In [28]:
on, ontable = reader.problem.language.get('on', 'ontable')
G = [clear(a), on(a, b), on(b, c), on(c, d), ontable(d)]

In [29]:
K_G = []
for l_G in G:
    p_l_G = KP_lang.get(str(l_G))
    K_G += [K(p_l_G)]

### Constructing the action set $O'$

In [30]:
from tarski.syntax.transform.action_grounding import ground_schema

In [31]:
O = []
for name, ops in actions.items():
    print('Action schema', name, 'got', len(ops), 'ground actions')
    schema = reader.problem.get_action(name)
    print(list(schema.parameters.vars()))
    for op in ops:
        ground_action = ground_schema(schema, op)
        O += [ground_action]
        print(ground_action)
        print(ground_action.precondition, ground_action.effects)

Action schema pick-up got 4 ground actions
[?x (object)]
pick-up(b)()
(clear(b) and ontable(b) and handempty()) [(T -> DEL(ontable(b))), (T -> DEL(clear(b))), (T -> DEL(handempty())), (T -> ADD(holding(b)))]
pick-up(c)()
(clear(c) and ontable(c) and handempty()) [(T -> DEL(ontable(c))), (T -> DEL(clear(c))), (T -> DEL(handempty())), (T -> ADD(holding(c)))]
pick-up(a)()
(clear(a) and ontable(a) and handempty()) [(T -> DEL(ontable(a))), (T -> DEL(clear(a))), (T -> DEL(handempty())), (T -> ADD(holding(a)))]
pick-up(d)()
(clear(d) and ontable(d) and handempty()) [(T -> DEL(ontable(d))), (T -> DEL(clear(d))), (T -> DEL(handempty())), (T -> ADD(holding(d)))]
Action schema put-down got 4 ground actions
[?x (object)]
put-down(b)()
holding(b) [(T -> DEL(holding(b))), (T -> ADD(clear(b))), (T -> ADD(handempty())), (T -> ADD(ontable(b)))]
put-down(c)()
holding(c) [(T -> DEL(holding(c))), (T -> ADD(clear(c))), (T -> ADD(handempty())), (T -> ADD(ontable(c)))]
put-down(a)()
holding(a) [(T -> DEL(hol

In [32]:
import copy
from tarski.syntax.transform.substitutions import substitute_subformula
from tarski.fstrips import AddEffect, DelEffect

In [33]:
K_O = []

subst = {}
for p, Kp in zip(F_lits, [K(p) for p in P_lits]):
    subst[symref(p)] = Kp

In [34]:
def make_K_condition_and_effect(eff, K_prec):
    KC = [K_prec]
    if not isinstance(eff.condition, Tautology):
        KC += [substitute_subformula(copy.deepcopy(eff.condition), subst)]
    KC = land(*KC)
    KL = subst[symref(eff.atom)]
    
    return KC, KL    

for op in O:
    K_prec = substitute_subformula(copy.deepcopy(op.precondition), subst)
    K_effs = []
    for eff in op.effects:
        KC, KL = make_K_condition_and_effect(eff, K_prec)
        if isinstance(eff, AddEffect):
            K_effs += [AddEffect(KC, KL)]
        elif isinstance(eff, DelEffect):
            K_effs += [DelEffect(KC, KL)]
        else:
            raise RuntimeError("Effect type not supported by compilation!")
        
